# Importing Libraries

In [13]:
import pandas as pd
import numpy as np
import re
import string

import codecs

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn import feature_extraction
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import warnings
warnings.simplefilter("ignore")

In [14]:
train = pd.read_csv(r"C:\Users\admin\Documents\EXPLORE\Advanced Classification\Exam\south-african-language-identification-hack-2023\train_set.csv")
test = pd.read_csv(r"C:\Users\admin\Documents\EXPLORE\Advanced Classification\Exam\south-african-language-identification-hack-2023\test_set.csv")

In [15]:
df = train.copy()
dftest = test.copy()

In [16]:
df.head(8)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
5,nso,dinyakišišo tše tša go dirwa gabedi ka ngwaga ...
6,tsn,kgetse nngwe le nngwe e e sa faposiwang mo tsh...
7,ven,mbadelo dze dza laelwa dzi do kwama mahatulele...


In [17]:
dftest.head(20)

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
5,6,"Ke feela dilense tše hlakilego, tša pono e tee..."
6,7,<fn>(762010101403 AM) 1495 Final Gems Birthing...
7,8,Ntjhafatso ya konteraka ya mosebetsi: Etsa bon...
8,9,u-GEMS uhlinzeka ngezinzuzo zemithi yezifo ezi...
9,10,"So, on occasion, are statistics misused."


In [18]:
df.shape, dftest.shape

((33000, 2), (5682, 2))

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [20]:
df["lang_id"].value_counts()

lang_id
xho    3000
eng    3000
nso    3000
ven    3000
tsn    3000
nbl    3000
zul    3000
ssw    3000
tso    3000
sot    3000
afr    3000
Name: count, dtype: int64

# Feature Seperation
 

# Text Pre-processing 

Removing any noise in our dataset

In [21]:
## Remove urls
print ('Removing URLs...')
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['text'] = df['text'].replace(to_replace = pattern_url, value = subs_url, regex = True)

# Make lower case
print ('Lowering case...')
df['text'] = df['text'].str.lower()

# Remove punctuation
import string
print ('Cleaning punctuation...')
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])
df['text'] = df['text'].apply(remove_punctuation_numbers)

Removing URLs...
Lowering case...
Cleaning punctuation...


In [22]:
X = df['text']
y = df['lang_id']

In [23]:
vector = TfidfVectorizer(sublinear_tf=True)
vector = TfidfVectorizer(ngram_range=(1,2), min_df=2)
X =vector.fit_transform(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,  random_state = 56)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0125, random_state=48)

In [25]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

lr_prediction = lr.predict(X_test)
lr_accuracy = accuracy_score(y_test,lr_prediction)

print("Training accuracy Score    : ",lr.score(X_train,y_train))
print("Testing accuracy Score : ",lr_accuracy )
print(classification_report(lr_prediction,y_test))

Training accuracy Score    :  0.9994318181818181
Testing accuracy Score :  0.9965151515151515
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       611
         eng       1.00      1.00      1.00       587
         nbl       0.99      0.99      0.99       607
         nso       1.00      1.00      1.00       594
         sot       1.00      1.00      1.00       576
         ssw       1.00      1.00      1.00       643
         tsn       1.00      1.00      1.00       613
         tso       1.00      1.00      1.00       607
         ven       1.00      1.00      1.00       597
         xho       1.00      0.98      0.99       588
         zul       0.98      0.99      0.99       577

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



# Improving a Logistc Model

In [26]:
# Import the scaler module
from sklearn import preprocessing
scaler = preprocessing.MaxAbsScaler()

# Scale data
X_scaled = scaler.fit_transform(X)

In [27]:
# Split into train and test
X1_train, X1_test, y1_train, y1_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

In [28]:
lm_full = LogisticRegression()

In [29]:
# No transformations necessary
lm_full.fit(X1_train, y1_train)

LogisticRegression()

In [30]:
# Generate predictions from full model
pred_lm_full = lm_full.predict(X1_test)

lm_full_accuracy = accuracy_score(y1_test,pred_lm_full)

print("Training accuracy Score    : ",lm_full.score(X1_train,y1_train))
print("Testing accuracy Score : ",lm_full_accuracy )
print(classification_report(pred_lm_full,y1_test))

Training accuracy Score    :  1.0
Testing accuracy Score :  0.9966666666666667
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       572
         eng       1.00      1.00      1.00       612
         nbl       0.98      1.00      0.99       559
         nso       1.00      1.00      1.00       574
         sot       1.00      1.00      1.00       567
         ssw       1.00      0.99      0.99       648
         tsn       1.00      1.00      1.00       587
         tso       1.00      1.00      1.00       615
         ven       1.00      1.00      1.00       624
         xho       1.00      1.00      1.00       606
         zul       0.99      0.99      0.99       636

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [33]:
y3 = df['lang_id']
y3 = y3.iloc[:5682]
test2 = dftest['text']
test2 = test2.iloc[:5682]
X3 = test2
X1_test =vector.transform(X3)

test_pred = lm_full.predict(X1_test)

y3 = test_pred
test['lang_id'] = test_pred

submit = test[['index','lang_id']]
submit.to_csv('C:\\Users\\admin\\Desktop\\Hackathon\\submit\\submitlogisticReg3.csv', index=False)


In [34]:
submit

,index,lang_id
0,1,ssw
1,2,nbl
2,3,xho
3,4,ssw
4,5,ssw
...,...,...
5677,5678,ssw
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [35]:
NB = MultinomialNB()

NB.fit(X_train,y_train)

NB_prediction = NB.predict(X_test)

NB_accuracy = accuracy_score(y_test,NB_prediction)

print("training accuracy Score: ", NB.score(X_train,y_train))
print("Testing accuracy Score : ", NB_accuracy )

print(classification_report(y_test,NB_prediction))

training accuracy Score:  0.9995833333333334
Testing accuracy Score :  0.9986363636363637
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       611
         eng       1.00      1.00      1.00       587
         nbl       1.00      0.99      1.00       611
         nso       1.00      1.00      1.00       594
         sot       1.00      1.00      1.00       576
         ssw       1.00      1.00      1.00       641
         tsn       1.00      1.00      1.00       613
         tso       1.00      1.00      1.00       607
         ven       1.00      1.00      1.00       597
         xho       1.00      1.00      1.00       579
         zul       0.99      1.00      1.00       584

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [37]:
test1 = test['text']
X_test =vector.transform(test1)

test_pred = NB.predict(X_test)

test['lang_id'] = test_pred

submit = test[['index','lang_id']]
submit.to_csv(submit.to_csv('C:\\Users\\admin\\Desktop\\Hackathon\\submit\\submitNB.csv', index=False)
, index=False) 

'index,lang_id\r\n1,tsn\r\n2,nbl\r\n3,ven\r\n4,ssw\r\n5,afr\r\n6,nso\r\n7,eng\r\n8,sot\r\n9,zul\r\n10,eng\r\n11,nso\r\n12,tso\r\n13,tsn\r\n14,sot\r\n15,tso\r\n16,xho\r\n17,xho\r\n18,ssw\r\n19,afr\r\n20,ven\r\n21,afr\r\n22,ven\r\n23,zul\r\n24,afr\r\n25,nso\r\n26,zul\r\n27,ven\r\n28,afr\r\n29,nso\r\n30,tso\r\n31,zul\r\n32,xho\r\n33,sot\r\n34,nso\r\n35,ssw\r\n36,afr\r\n37,nso\r\n38,xho\r\n39,sot\r\n40,eng\r\n41,zul\r\n42,ssw\r\n43,ven\r\n44,nbl\r\n45,afr\r\n46,ven\r\n47,zul\r\n48,ven\r\n49,nbl\r\n50,tsn\r\n51,sot\r\n52,xho\r\n53,sot\r\n54,sot\r\n55,zul\r\n56,ven\r\n57,nbl\r\n58,xho\r\n59,ssw\r\n60,sot\r\n61,afr\r\n62,tso\r\n63,afr\r\n64,zul\r\n65,xho\r\n66,ven\r\n67,sot\r\n68,nso\r\n69,nso\r\n70,zul\r\n71,tsn\r\n72,tso\r\n73,xho\r\n74,nso\r\n75,afr\r\n76,ven\r\n77,zul\r\n78,tso\r\n79,xho\r\n80,ssw\r\n81,tsn\r\n82,nso\r\n83,xho\r\n84,tsn\r\n85,afr\r\n86,nso\r\n87,sot\r\n88,tso\r\n89,tso\r\n90,xho\r\n91,ssw\r\n92,xho\r\n93,zul\r\n94,sot\r\n95,ssw\r\n96,nso\r\n97,afr\r\n98,ssw\r\n99,ven\r\n1

In [38]:
submit

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr
...,...,...
5677,5678,eng
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [39]:
rf = RandomForestClassifier()

rf.fit(X_train,y_train)

rf_prediction = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test,rf_prediction)

print("Training accuracy Score    : ",rf.score(X_train,y_train))
print("Testing accuracy Score : ",rf_accuracy )

print(classification_report(rf_prediction,y_test))

ValueError: Found input variables with inconsistent numbers of samples: [6600, 5682]